# Objective: Match Kravtsov's BCGs to objects in RedMaPPer

* Kravtsov (2014) considers a sample of nine Brightest Cluster Galaxies, comparing their stellar mass estimates with those of Bernardi et al.


* In this notebook, we first use the `astropy` function `match_catalog_to_sky` to find the overlap between Kravtsov's BCGs and our `RedMaPPer` sample. The RA/Dec of the sources from Kravtsov were hand-picked from SDSS DR13. Our finalized coordinates are available on our GutHub in an `ecsv` file.

    * <a href="http://skyserver.sdss.org/dr13/en/tools/chart/navi.aspx" target="_blank">SDSS DR13</a>
    * <a href="http://legacysurvey.org/viewer" target="_blank">DECaLS Sky Viewer</a>
    * <a href="https://github.com/moustakas/siena-astrophysics/blob/masscompare/research/massivepilot/kravsources.ecsv" target="_blank">Finalized Coordinates</a>
    
    * <a href="https://yymao.github.io/decals-image-list-tool/" target="_blank">Make Some Quick Cutouts</a>


* We then compare the stellar masses (M*) of the matching sources.

In [1]:
%%html
<center><img src="https://omundy.files.wordpress.com/2012/04/i-will-not-write-any-more-bad-code.gif" 
alt="Jokes"></center>

In [2]:
import os

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
from astropy.io.ascii import read
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
import fitsio
import seaborn

## Reading in all necessary data and making initial cuts.

In [5]:
# Setting convenient paths.
pypath = os.path.join(os.sep, 'global', 'work', 'projects', 'legacysurvey', 'legacycentrals')
meertcat = os.path.join(pypath, 'meert_et_al_data_tables_v2')
mendelcat = os.path.join(pypath, 'UPenn_PhotDec_Mstar_mlMendel14.dat')
wisccat = os.path.join(pypath, 'UPenn_PhotDec_Mstar_mlPCAWiscM11.dat')

In [6]:
# Read the Mendel catalog
columns = ('GalCount', 'FlagSerExp', 'Mstar_Tab5_Pymorph',
           'Mstar_Tab5_Truncated', 'Mstar_Tab3_Pymorph',
           'Mstar_Tab3_Truncated', 'Mstar_Tab5_Mendel',
           'Mstar_Tab3_Mendel', 'Mstar_Tab5_cModel',
           'Mstar_Tab3_cModel')
dtype = np.dtype([(col, np.float) for col in columns])
allmendel = np.loadtxt(mendelcat, dtype=dtype)

In [7]:
# Trim the Mendel catalog. Here we simply remove the bad flags.
keep = (np.where(allmendel['FlagSerExp'] == 0))[0]
print('Keeping {} / {} measurements in the Mendel catalog.'.format(len(keep), len(allmendel)))
mendel = allmendel[keep]

Keeping 628919 / 670722 measurements in the Mendel catalog.


In [8]:
# Read the parent Meert catalog to get ra, dec and other info.
upennpath = os.path.join(pypath, 'meert_et_al_data_tables_v2')
upenncatfile = os.path.join(upennpath, 'UPenn_PhotDec_CAST.fits')
upenncat = fitsio.read(upenncatfile, ext=1, rows=keep, upper=True)

In [9]:
# Reading in all the RedMaPPer iSEDfit catalog.
rmpath = os.path.join(os.sep, 'global', 'work', 'projects', 'redmapper')
rmcatfile = os.path.join(rmpath, 'redmapper_isedfit_v5.10_centrals.fits.gz')
rminfo = fitsio.FITS(rmcatfile)
rmcat = rminfo[1].read(columns=['MEM_MATCH_ID', 'Z', 'RA', 'DEC', 'MSTAR_50', 'MSTAR_AVG', 'MSTAR_ERR'])

In [10]:
matched = os.path.join(os.sep, 'home', 'kjnapes')
mcentrals = os.path.join(matched, 'tractor_redmapper_isedfit_matches_centrals.fits')
matchedinfo = fitsio.FITS(mcentrals)
matchedcat = matchedinfo[1].read(columns=['RA', 'DEC'])

In [11]:
#rcoord = SkyCoord(ra=matchedcat['RA']*u.degree, dec=matchedcat['DEC']*u.degree)
#rmcoord = SkyCoord(ra=rmcat['RA']*u.degree, dec=rmcat['DEC']*u.degree)
#idx, sep2d, dist3d = rcoord.match_to_catalog_sky(rmcoord, nthneighbor=1)

In [12]:
#gd = np.where(sep2d < 0.001 * u.arcsec)[0]
#print(len(gd))

In [13]:
# WARNING! This cell is huge. It takes a long time to run.
satellites = os.path.join(rmpath, 'redmapper_isedfit_v5.10_satellites.fits.gz')
satinfo = fits.open(satellites)

In [14]:
#rcoord = SkyCoord(ra=matchedcat['RA']*u.degree, dec=matchedcat['DEC']*u.degree)
#rmcoord = SkyCoord(ra=satinfo[1].data['RA']*u.degree, dec=satinfo[1].data['DEC']*u.degree)
#idx, sep2d, dist3d = rcoord.match_to_catalog_sky(rmcoord, nthneighbor=1)

In [15]:
#gd = np.where(sep2d < 0.001 * u.arcsec)[0]
#print(len(gd))

In [16]:
rcoord = SkyCoord(ra=satinfo[1].data['RA']*u.degree, dec=satinfo[1].data['DEC']*u.degree)
rmcoord = SkyCoord(ra=rmcat['RA']*u.degree, dec=rmcat['DEC']*u.degree)
idx, sep2d, dist3d = rcoord.match_to_catalog_sky(rmcoord, nthneighbor=1)

In [17]:
gd = np.where(sep2d < 0.001 * u.arcsec)[0]
#print(satinfo[1].data['RA'][gd]-rmcat['RA'][idx[gd]])

In [18]:
overlap=open("overlap.txt","w")
for val in zip(rmcat['MEM_MATCH_ID'][idx[gd]], rmcat['RA'][idx[gd]],rmcat['DEC'][idx[gd]]):
    overlap.write('{}, {}, {}\n'.format(str(val[0]), val[1], val[2]))
overlap.close()

In [19]:
# Reading in the RA, Dec, and helioZ of Kravtsov's selected objects.
kravsources = os.path.join(os.sep, 'home','kjnapes', 'siena-astrophysics', 'research', 'massivepilot',
                           'kravsources.txt')
sourceRA, sourceDEC = np.loadtxt(kravsources, unpack=True, usecols=(1,2))
name = np.genfromtxt(kravsources, dtype='U', usecols=0)

In [20]:
#rcoord = SkyCoord(ra=satinfo[1].data['RA']*u.degree, dec=satinfo[1].data['DEC']*u.degree)
#kravcoord = SkyCoord(ra=sourceRA*u.degree, dec=sourceDEC*u.degree)
#idx, sep2d, dist3d = kravcoord.match_to_catalog_sky(rcoord, nthneighbor=1)

In [21]:
# Applying a limiting tolerance to matches. 30 arcseconds is a reasonable radius.
gd = np.where(sep2d < 3 * u.arcsec)[0]
print(len(gd))
print(name[gd])
print(satinfo[1].data['RA'][idx[gd]])
print(satinfo[1].data['DEC'][idx[gd]])
print(satinfo[1].data['Z'][idx[gd]]) # The redshifts are a little off...
print(satinfo[1].data['MSTAR_50'][idx[gd]])
print(satinfo[1].data['MSTAR_AVG'][idx[gd]])
print(satinfo[1].data['MSTAR_ERR'][idx[gd]])

20733


IndexError: index 502 is out of bounds for axis 1 with size 9

## Matching RedMaPPer and Kravtsov

In [ ]:
# Cross-matching catalogs
rcoord = SkyCoord(ra=rmcat['RA']*u.degree, dec=rmcat['DEC']*u.degree)
kravcoord = SkyCoord(ra=sourceRA*u.degree, dec=sourceDEC*u.degree)
idx, sep2d, dist3d = kravcoord.match_to_catalog_sky(rcoord, nthneighbor=1)

# Applying a limiting tolerance to matches. 30 arcseconds is a reasonable radius.
gd = np.where(sep2d < 30 * u.arcsec)[0]

In [ ]:
# Looking at the indices and properties of the matches. 
print(gd)
print(name[gd])
print(rmcat['RA'][idx[gd]])
print(rmcat['DEC'][idx[gd]])
print(rmcat['Z'][idx[gd]]) # The redshifts are a little off...
print(rmcat['MSTAR_50'][idx[gd]])
print(rmcat['MSTAR_AVG'][idx[gd]])
print(rmcat['MSTAR_ERR'][idx[gd]])

In [ ]:
# Generate a custom FITS file for matching purposes
from astropy.table import Table, Column
out = Table()
out.add_column(Column(name='RA', data=rmcat['RA'][idx[gd]]))
out.add_column(Column(name='DEC', data=rmcat['DEC'][idx[gd]]))
out.write(os.path.join(os.sep, 'home','kjnapes', 'siena-astrophysics', 'research', 'massivepilot',
                           'rmmatches.fits'), overwrite=True) # Clobber is deprecated

## Matching PyMorph and Kravtsov

In [ ]:
# Cross-matching catalogs
rcoord = SkyCoord(ra=upenncat['RA']*u.degree, dec=upenncat['DEC']*u.degree)
kravcoord = SkyCoord(ra=sourceRA*u.degree, dec=sourceDEC*u.degree)
idx, sep2d, dist3d = kravcoord.match_to_catalog_sky(rcoord, nthneighbor=1)

# Applying a limiting tolerance to matches
gdpy = np.where(sep2d < 30 * u.arcsec)[0]
print(len(gdpy))

In [ ]:
print(name[gdpy])
print(upenncat['RA'][idx[gdpy]])
print(upenncat['DEC'][idx[gdpy]])

# Looking at the overlap between the catalogs.

In [ ]:
fig = plt.figure(figsize(18,14))

ax2 = fig.add_subplot(221)
ax2.scatter(rmcat['RA'], rmcat['DEC'], color='gray', label='RedMaPPer')
ax2.scatter(sourceRA, sourceDEC, alpha=0.9, color='white', label='Kravtsov(full)')
ax2.legend(loc='upper left')

ax1 = fig.add_subplot(222)
ax1.scatter(rmcat['RA'], rmcat['DEC'], color='gray', label='RedMaPPer')
ax1.scatter(sourceRA[gd], sourceDEC[gd], alpha=0.9, color='white', label='Kravtsov (matched)')
ax1.legend(loc='upper left')

ax3 = fig.add_subplot(223)
ax3.scatter(upenncat['RA'], upenncat['DEC'], color='black', label='PyMorph')
ax3.scatter(sourceRA, sourceDEC, alpha=0.9, color='white', label='Kravtsov (full)')
ax3.legend(loc='upper left')

ax4 = fig.add_subplot(224)
ax4.scatter(upenncat['RA'], upenncat['DEC'], color='black', label='PyMorph')
ax4.scatter(sourceRA[gdpy], sourceDEC[gdpy], alpha=0.9, color='white', label='Kravtsov (matched)')
ax4.legend(loc='upper left')

In [ ]:
from IPython.display import IFrame
IFrame('cutouts.html', width=700, height=1000)

## Creating a fits file of matched redMaPPer/PyMorph, along with $\lambda$, $M_{500}$, $M_{*,Moustakas}$, $M_{*, PyMorph}$.

In [26]:
# Read the parent Meert catalog to get ra, dec and other info.
upenncatfile = os.path.join(meertcat, 'UPenn_PhotDec_CAST.fits')
upenncat = fitsio.read(upenncatfile, ext=1, rows=keep, upper=True)

In [33]:
rmpath = os.path.join(os.sep, 'global', 'work', 'projects', 'redmapper')
rmcatfile = os.path.join(rmpath, 'redmapper_isedfit_v5.10_centrals.fits.gz')
rminfo = fitsio.FITS(rmcatfile)
rmcat = rminfo[1].read(columns=['Z', 'RA', 'DEC', 'LAMBDA_CHISQ', 'MSTAR_50', 'MSTAR_ERR', 'IMAG'])

In [34]:
# Cross-matching RedMaPPer and PyMorph
rcoord = SkyCoord(ra=rmcat['RA']*u.degree, dec=rmcat['DEC']*u.degree)
upenn = SkyCoord(ra=upenncat['RA']*u.degree, dec=upenncat['DEC']*u.degree)
idx, sep2d, dist3d = rcoord.match_to_catalog_sky(upenn, nthneighbor=1)

In [35]:
# Applying a limiting tolerance to matches
gd = np.where(sep2d < 3 * u.arcsec)[0]
print(len(gd))

19760


In [ ]:
from astropy.table import Table, Column
out = Table()
out.add_column(Column(name='LAMBDA', data=rmcat['LAMBDA_CHISQ'][idx[gd]]))
out.add_column(Column(name='DEC', data=rmcat['DEC'][idx[gd]]))
out.write(os.path.join(os.sep, 'home','kjnapes', 'siena-astrophysics', 'research', 'massivepilot',
                           'rmmatches.fits'), overwrite=True) # Clobber is deprecated